In [1]:
import numpy as np
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers

# VGG16 모델 로드 (include_top=False로 특징 추출용으로 사용)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# 모델 구성
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # 기존 Dense 레이어
    Dropout(0.5),
    Dense(128, activation='relu'),  # 추가 Dense 레이어
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

# 옵티마이저 변경 (학습률 동적 조정)
optimizer = Adam(learning_rate=0.0001)
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

# 초기 학습률 높게 설정 후 점차 감소
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# 데이터 증강
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,  # 회전 범위 적당히 추가
    width_shift_range=0.1,  # 가로 이동 범위
    height_shift_range=0.1,  # 세로 이동 범위
    zoom_range=0.1,  # 확대 범위
    horizontal_flip=True
)

# train set
train_generator = train_datagen.flow_from_directory(
    './data/train_added/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

# valid set
valid_generator = train_datagen.flow_from_directory(
    './data/validation-horse-or-human/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

# 콜백 설정
es = EarlyStopping(
    monitor='val_loss',
    patience=10,  # patience 값을 10으로 증가
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath='./best_model3.keras',
    monitor='val_loss',
    verbose=2,
    save_best_only=True,
    mode='auto'
)

# 학습률 조정 콜백 (ReduceLROnPlateau)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# steps_per_epoch 설정
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    epochs=30,
    verbose=1,
    callbacks=[checkpoint, es, reduce_lr]
)


Found 2890 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/30


c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.7508 - loss: 0.8466 
Epoch 1: val_loss improved from inf to 0.49457, saving model to ./best_model3.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 730s 16s/step - accuracy: 0.7532 - loss: 0.8422 - val_accuracy: 0.9219 - val_loss: 0.4946 - learning_rate: 1.0000e-04
Epoch 2/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/30


c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\callback_list.py:96: UserWarning: Learning r

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.9862 - loss: 0.3803 
Epoch 3: val_loss did not improve from 0.49457
46/46 ━━━━━━━━━━━━━━━━━━━━ 737s 16s/step - accuracy: 0.9863 - loss: 0.3800 - val_accuracy: 0.8828 - val_loss: 0.5506 - learning_rate: 1.0000e-04
Epoch 4/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 391us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 5/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.9834 - loss: 0.3405 
Epoch 5: val_loss improved from 0.49457 to 0.39785, saving model to ./best_model3.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 782s 17s/step - accuracy: 0.9835 - loss: 0.3404 - val_accuracy: 0.9414 - val_loss: 0.3979 - learning_rate: 1.0000e-04
Epoch 6/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 7/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.9865 - loss: 0.3122 
Epoch 7: val_loss did not improve from 0.39785
46/46 ━━━━━━━━━━━━━━━━━━━━ 707s 15s/step

KeyboardInterrupt: 

In [ ]:
# 모델 검증
results = model.evaluate(valid_generator)
print("test loss, test acc:", results)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Model accuracy & loss')
plt.xlabel('Epoch')
plt.ylabel('value')
plt.legend(['accuracy', 'loss'], loc='center right')
plt.show()